<a href="https://colab.research.google.com/github/guruu27/CAPSTONE-PROJECT/blob/main/torontosegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PART 1





In [29]:
import numpy as np
import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

 # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


In [30]:
#lxml will be the foundation for pd.read_html()
import lxml
from lxml import etree

from urllib.request import urlopen
import html5lib

#Lets get some soup to help with scrapping HTML URL
from bs4 import BeautifulSoup


In [31]:
url= "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969"
toronto_hood_codes = pd.read_html(url, match= "Neighbourhood")

In [32]:
df=pd.DataFrame(toronto_hood_codes[0])
df.replace('Not assigned', np.NaN, inplace=True )
df.dropna(inplace=True)

In [33]:
df.reset_index(drop=True, inplace=True)

In [34]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# **PART 2**

In [35]:
#download the data and open the download data 
read = requests.get("http://cocl.us/Geospatial_data", allow_redirects=True)
open('data.csv', 'wb').write(read.content)
data = pd.read_csv('data.csv')
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
#merge the data base on postal code
df = df.merge(data, on= 'Postal Code')
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [50]:
df['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Toronto/York', 'Mississauga'], dtype=object)

# **PART 3**

In [38]:
df.groupby('Borough').count()

,Postal Code,Neighbourhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Scarborough,17,17,17,17
Toronto/York,1,1,1,1


In [46]:
#dropping the group of Borough and postal code
X = df.drop(['Borough','Postal Code'], axis =1)


In [47]:
k=10
df_clustering=X.drop('Neighbourhood',axis=1)
kmeans=KMeans(n_clusters=k,random_state=1).fit(df_clustering)
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=1, tol=0.0001, verbose=0)

In [48]:
X.insert(3,'Cluster Labels', kmeans.labels_)
X.head()

,Neighbourhood,Latitude,Longitude,Cluster Labels
0,Parkwoods,43.753259,-79.329656,2
1,Victoria Village,43.725882,-79.315572,6
2,"Regent Park, Harbourfront",43.654260,-79.360636,0
3,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4
4,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0


In [49]:
#create the map
map_clusters = folium.Map(location=[X['Latitude'][0], X['Longitude'][0]])

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(X['Latitude'], X['Longitude'], X['Neighbourhood'], X['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius= 10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters